<a href="https://colab.research.google.com/github/djmayo04/AI_Mini-Project2/blob/main/notebooks/AI_MP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U langchain langchain-groq groq

  Using cached groq-1.0.0-py3-none-any.whl.metadata (16 kB)


In [2]:
import os

REPO_URL = "https://github.com/djmayo04/AI_Mini-Project2.git"
REPO_DIR = "AI_Mini-Project2"

%cd /content

if not os.path.exists(REPO_DIR):
    !git clone {REPO_URL}
else:
    %cd {REPO_DIR}
    !git pull
    %cd /content

%cd {REPO_DIR}
!ls

/content
/content/AI_Mini-Project2
Already up to date.
/content
/content/AI_Mini-Project2
agent.py   data		  memory.py  __pycache__  search_documents.py
chroma_db  llm_helper.py  notebooks  README.md	  tools.py


In [3]:
import os
from google.colab import userdata
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')
print(os.environ.get("GROQ_API_KEY") is not None)

True


In [4]:
!pip install --upgrade pip
!pip install -U langchain langchain-community langchain-text-splitters
!pip install chromadb

In [5]:
# load the data by looping through each file and split it into chunks with langchain_text_splitters done in llmhelper.py
from langchain_community.document_loaders import TextLoader
from llm_helper import chunk_split

all_docs = []

for file in [
    "data/Assignment1.txt",
    "data/Rubric.txt",
    "data/Syllabus.txt",
    "data/faq.txt"
]:
    loader = TextLoader(file)
    all_docs.extend(loader.load())

chunks = chunk_split(all_docs)

In [6]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

# Embedding model
embeddings = HuggingFaceEmbeddings()

# Transforms chunks into vectors for storage using HuggingFaceEmbeddings model
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="./chroma_db"
)


/tmp/ipython-input-182361187.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
/tmp/ipython-input-182361187.py:5: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

MPNetModel LOAD REPORT from: sentence-transformers/all-mpnet-base-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [7]:
from tools import set_vectorstore

# Set/export the vectorstore to tools.py

set_vectorstore(vectorstore)

In [8]:
# Test of similarity_search in the vectorstore
docs = vectorstore.similarity_search("assignment deadline", k=2)
print(docs[0].page_content)

Frequently Asked Questions – AI Course

Q: When is Assignment 1 due?
A: February 25 at 11:59 PM.

Q: Is late submission allowed?
A: Yes, within 48 hours with penalty.

Q: When is the midterm exam?
A: March 10.

Q: What percentage is the final project worth?
A: 30% of the final grade.

Q: Can I work in teams for Assignment 1?
A: No, Assignment 1 is individual.

Q: Can I use external libraries?
A: Yes, but you must explain them in your report.


In [33]:
from llm_helper import get_completion
import agent

print(get_completion("Say hello in one sentence."))
print(run_agent("When is assignment 1 due?"))
print(agent.memory.get_history())

Hello! Hope you're having a wonderful day.
I don’t have access to the specific details of your course schedule. The best way to find out the due date for Assignment 1 is to check your syllabus, the course’s online portal (e.g., Canvas, Blackboard, Moodle), or ask your instructor or teaching assistant directly. If you have any other questions or need help with the assignment itself, feel free to let me know!
[{'role': 'user', 'content': 'When is assignment 2 due?'}, {'role': 'assistant', 'content': 'I’m not sure which class or course you’re referring to. Could you let me know the name of the course (or any other details you have about the assignment) so I can give you the correct due‑date information?'}, {'role': 'user', 'content': 'When is assignment 1 due?'}, {'role': 'assistant', 'content': 'I don’t have access to the specific details of your course schedule. The best way to find out the due date for Assignment\u202f1 is to check your syllabus, the course’s online portal (e.g., Canva

In [37]:
print(run_agent("When is assignment 2 due?"))
print(agent.memory.get_history())

Assignment 2 is due on **March 20**.
[{'role': 'user', 'content': 'When is assignment 2 due?'}, {'role': 'assistant', 'content': 'I’m not sure which course you’re referring to—could you let me know the class or subject (e.g., Math\u202f101, English\u202f202, etc.)? Then I can give you the correct due date for Assignment\u202f2.'}, {'role': 'user', 'content': 'When is assignment 2 due?'}, {'role': 'assistant', 'content': 'I don’t have access to the specific details for your class, but you can usually find the due date for Assignment\u202f2 in one of the following places:\n\n- **Course syllabus** (often posted on the first day of class or on the course website)\n- **Learning management system** (e.g., Canvas, Blackboard, Moodle) under the “Assignments” or “Modules” section\n- **Instructor or TA announcements** (email, discussion board, or class forum)\n- **Course calendar** (if your school provides a centralized academic calendar)\n\nIf you still can’t locate it, I recommend reaching out